In [37]:
from langchain_community.vectorstores import FAISS
import os
from pyprojroot import here
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

True

In [38]:
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

# Configure models and paths
EMBEDDING_MODEL = "models/embedding-001"  # Google's text embedding model
VECTORDB_DIR = "data/airline_policy_vectordb"  # Path to your existing FAISS database
K = 2 

In [39]:
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")
if not os.environ.get('GOOGLE_API_KEY'):
    print("WARNING: GOOGLE_API_KEY environment variable is not set!")


In [40]:
embedding_function = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)

In [41]:
# instantiate your embedding function
embedding_function = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)

try:
    vectordb = FAISS.load_local(
        folder_path=str(here(VECTORDB_DIR)),
        embeddings=embedding_function,
        allow_dangerous_deserialization=True  # <— tell FAISS it’s OK
    )
    print("Successfully loaded FAISS index")
    print("Number of vectors in vectordb:", len(vectordb.index_to_docstore_id))
except Exception as e:
    print(f"Error loading FAISS index: {e}")
    raise


Successfully loaded FAISS index
Number of vectors in vectordb: 22


In [42]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"
docs = vectordb.similarity_search(message, k=K)
docs

[Document(id='46e6832b-a83c-41c2-bf9f-45a1d9019b9e', metadata={'producer': 'PDFCreator Online (www.pdfforge.org/online)', 'creator': 'PDFCreator Online (www.pdfforge.org/online)', 'creationdate': '2024-09-08T21:34:59+00:00', 'title': 'Merged with PDFCreator Online', 'moddate': '2024-09-08T21:34:59+00:00', 'source': 'C:\\Users\\Lenovo\\LLM Project\\SQLAgent-L-L-\\data\\unstructured_docs\\swiss_airline_policy\\swiss_faq.pdf', 'total_pages': 12, 'page': 8, 'page_label': '9'}, page_content="How to Cancel a Swiss Air Flight: 877-\n5O7-7341 Step-by-Step Guide\nSwiss Air is a premium airline based in Switzerland that offers a range of domestic and international flights to\npassengers. However, sometimes situations arise where passengers may need to cancel their flights. In such cases, it is\nimportant to understand the Swiss Air Cancellation Policy to avoid any confusion or additional charges.\nSwiss International Airlines Cancellation Policy In this article, we will provide you with everythi

In [43]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in docs:
    retrieved_content += f"{doc.page_content}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

In [44]:
pprint(prompt)

('# Content:\n'
 'How to Cancel a Swiss Air Flight: 877-\n'
 '5O7-7341 Step-by-Step Guide\n'
 'Swiss Air is a premium airline based in Switzerland that offers a range of '
 'domestic and international flights to\n'
 'passengers. However, sometimes situations arise where passengers may need to '
 'cancel their flights. In such cases, it is\n'
 'important to understand the Swiss Air Cancellation Policy to avoid any '
 'confusion or additional charges.\n'
 'Swiss International Airlines Cancellation Policy In this article, we will '
 'provide you with everything you need to know about\n'
 'the Swiss Air Cancellation Policy, including how to cancel a Swiss Air '
 'flight, the fees associated with cancelling a flight,\n'
 'and the refund policy.\n'
 "If you have booked a flight with Swiss Airlines but need to cancel it, it's "
 'important to understand their cancellation policy\n'
 'to avoid any unnecessary fees or charges. Swiss Airlines offers different '
 'fare types, each with their own 

In [45]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  
    temperature=0
)

In [46]:
response = llm.invoke([
    ("system", "You will receive a user's query and possible content where the answer might be. If the answer is found, provide it, if not, state that the answer does not exist."),
    ("human", prompt)
])

print(response.content)

Swiss Airlines' cancellation policy varies depending on your fare type and the time of cancellation. The most flexible fare types such as Flex and Business Flex allow you to cancel your flight up to 24 hours before departure without any penalty. For other fare types, cancellation fees may apply. If you cancel your Swiss Airlines flight outside of the 24-hour window, cancellation fees will be charged depending on your fare type and the time of cancellation. For example, if you cancel a non-flexible economy class ticket, a cancellation fee will be charged. The closer you cancel to the departure date, the higher the cancellation fee.


In [49]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str) -> str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    # Load the vector database
    embedding_function = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)
    vectordb = FAISS.load_local(
        folder_path=str(here(VECTORDB_DIR)),
        embeddings=embedding_function,
        allow_dangerous_deserialization=True
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

# Test the tool
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

lookup_swiss_airline_policy
{'query': {'title': 'Query', 'type': 'string'}}
Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query.


In [50]:
pprint(lookup_swiss_airline_policy.invoke("can I cancel my ticket?"))

('Tickets with a status other than O (open) (A)\n'
 'Bookings with segments with a status other than OK (e.g. containing flight '
 'segments with the status\n'
 'Waitlist) (HK|RR)\n'
 'Tickets that do not display the tariff calculation (IT tickets)\n'
 'Bookings that contain special services (e.g. transportation of '
 'animals/transportation of medical\n'
 'equipment/transportation of special sporting equipment etc.)\n'
 'Tickets that are part of a group booking\n'
 'Tickets on which the booking class (RBD) does not correspond to the tariff '
 'code (FareBase)\n'
 '3. Am I allowed to make changes to my online redemption booking?\n'
 'Redemption bookings can not yet be changed on swiss.com.\n'
 '4. What changes can I make on my ticket?\n'
 'You can change the flight and the travel dates. Change of origin and '
 'destination or name changes are not\n'
 'possible online.\n'
 '5. After I have made changes to the itinerary online, can I make another '
 'change?\n'
 'If the fare condition al